In [1]:
#pip install mysql-connector-python
import mysql.connector
import csv

# Połączenie z serwerem MariaDB
conn = mysql.connector.connect(
    host="giniewicz.it",  # lub inny host, jeśli serwer jest zdalny
    user="team24",       # użytkownik MariaDB
    password="te@mzaza",  # hasło użytkownika
    database="team24",
    use_pure = True
)

if(conn):
    print("Połączenie udane")
else:
    print("Połączenie nieudane")

cursor = conn.cursor()

Połączenie udane


In [2]:
# Funkcja do usuwania wszystkich kluczy obcych w bazie danych, aby tabele się usuwały
def usun_klucze_obce():
    # Zapytanie, które znajduje wszystkie klucze obce w tabeli INFORMATION_SCHEMA
    cursor.execute("""
    SELECT CONSTRAINT_NAME, TABLE_NAME
    FROM INFORMATION_SCHEMA.KEY_COLUMN_USAGE
    WHERE CONSTRAINT_SCHEMA = DATABASE() AND REFERENCED_TABLE_NAME IS NOT NULL;
    """)
    
    foreign_keys = cursor.fetchall()

    # Iterujemy po wszystkich kluczach obcych i je usuwamy
    for fk in foreign_keys:
        constraint_name = fk[0]
        table_name = fk[1]
        
        # Usuwanie klucza obcego
        cursor.execute(f"ALTER TABLE {table_name} DROP FOREIGN KEY {constraint_name};")
        print(f"Usunięto klucz obcy: {constraint_name} z tabeli: {table_name}")

# Wywołanie funkcji usuwającej wszystkie klucze obce
usun_klucze_obce()

# Zatwierdzenie zmian
conn.commit()

Usunięto klucz obcy: FK_cennik_TO_oferty z tabeli: oferty
Usunięto klucz obcy: FK_kierunki_TO_oferty z tabeli: oferty
Usunięto klucz obcy: FK_miasta_TO_oferty z tabeli: oferty
Usunięto klucz obcy: FK_rodzaje_TO_oferty z tabeli: oferty
Usunięto klucz obcy: FK_oferty_TO_zorganizowana_wycieczka z tabeli: zorganizowana_wycieczka
Usunięto klucz obcy: FK_pracownicy_TO_urlopy z tabeli: urlopy
Usunięto klucz obcy: FK_osoby_TO_osoby_zorganizowana_wycieczka z tabeli: osoby_zorganizowana_wycieczka
Usunięto klucz obcy: FK_zorganizowana_wycieczka_TO_osoby_zorganizowana_wycieczka z tabeli: osoby_zorganizowana_wycieczka
Usunięto klucz obcy: FK_dane_alarmowe_TO_osoby z tabeli: osoby
Usunięto klucz obcy: FK_stanowiska_TO_pracownicy z tabeli: pracownicy
Usunięto klucz obcy: FK_pracownicy_TO_pracownicy_wycieczka z tabeli: pracownicy_wycieczka
Usunięto klucz obcy: FK_zorganizowana_wycieczka_TO_pracownicy_wycieczka z tabeli: pracownicy_wycieczka


In [3]:
# Usuwanie i utworzenie tabeli 'cennik'
cursor.execute("DROP TABLE IF EXISTS cennik;")
cursor.execute("""
CREATE TABLE cennik (
    id_cennika SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    koszt_na_osobe DECIMAL(9,2) NOT NULL COMMENT 'koszt jaki ponosimy za jedna osobę na wycieczce',
    cena_biletu DECIMAL(9,2) NOT NULL COMMENT 'cena jaka musi zapłacić klient',
    koszt_podstawowy DECIMAL(9,2) NOT NULL COMMENT 'koszt samej organizacji wydarzenia',
    PRIMARY KEY (id_cennika)
);
""")

cursor.execute("ALTER TABLE cennik ADD CONSTRAINT UQ_id_cennika UNIQUE (id_cennika);")

# Usuwanie i utworzenie tabeli 'dane_alarmowe'
cursor.execute("DROP TABLE IF EXISTS dane_alarmowe;")
cursor.execute("""
CREATE TABLE dane_alarmowe (
    id_alarm INT UNSIGNED NOT NULL AUTO_INCREMENT,
    imie VARCHAR(40) NULL,
    nazwisko VARCHAR(50) NULL,
    numer_telefonu VARCHAR(18) NULL,
    PRIMARY KEY (id_alarm)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE dane_alarmowe ADD CONSTRAINT UQ_id_alarm UNIQUE (id_alarm);")
cursor.execute("ALTER TABLE dane_alarmowe ADD CONSTRAINT UQ_numer_telefonu UNIQUE (numer_telefonu);")

# Usuwanie i utworzenie tabeli 'kierunki'
cursor.execute("DROP TABLE IF EXISTS kierunki;")
cursor.execute("""
CREATE TABLE kierunki (
    id_kierunku SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    miejsce VARCHAR(40) NULL,
    PRIMARY KEY (id_kierunku)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE kierunki ADD CONSTRAINT UQ_id_kierunku UNIQUE (id_kierunku);")

# Usuwanie i utworzenie tabeli 'miasta'
cursor.execute("DROP TABLE IF EXISTS miasta;")
cursor.execute("""
CREATE TABLE miasta (
    id_miasta SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    nazwa VARCHAR(40) NULL,
    PRIMARY KEY (id_miasta)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE miasta ADD CONSTRAINT UQ_id_miasta UNIQUE (id_miasta);")

# Usuwanie i utworzenie tabeli 'oferty'
cursor.execute("DROP TABLE IF EXISTS oferty;")
cursor.execute("""
CREATE TABLE oferty (
    id_oferty SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    nazwa VARCHAR(255) NULL,
    id_rodzaju SMALLINT UNSIGNED NOT NULL,
    id_kierunku SMALLINT UNSIGNED NOT NULL,
    id_miasta SMALLINT UNSIGNED NOT NULL,
    opis VARCHAR(255) NULL,
    id_cennika SMALLINT UNSIGNED NOT NULL,
    czas_trwania TINYINT UNSIGNED NOT NULL COMMENT 'Czas podawany w godzinach.',
    czy_bezpieczna BOOL NULL,
    PRIMARY KEY (id_oferty)
);
""")

cursor.execute("ALTER TABLE oferty ADD CONSTRAINT UQ_id_oferty UNIQUE (id_oferty);")
cursor.execute("ALTER TABLE oferty ADD CONSTRAINT UQ_id_cennika UNIQUE (id_cennika);")

# Usuwanie i utworzenie tabeli 'osoby'
cursor.execute("DROP TABLE IF EXISTS osoby;")
cursor.execute("""
CREATE TABLE osoby (
    id_osoby INT UNSIGNED NOT NULL AUTO_INCREMENT,
    imie VARCHAR(30) NULL,
    nazwisko VARCHAR(50) NULL,
    numer_telefonu VARCHAR(18) NULL,
    email VARCHAR(150) NULL,
    id_alarm INT UNSIGNED NOT NULL,
    PRIMARY KEY (id_osoby)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE osoby ADD CONSTRAINT UQ_id_osoby UNIQUE (id_osoby);")
cursor.execute("ALTER TABLE osoby ADD CONSTRAINT UQ_numer_telefonu UNIQUE (numer_telefonu);")
cursor.execute("ALTER TABLE osoby ADD CONSTRAINT UQ_email UNIQUE (email);")

# Usuwanie i utworzenie tabeli 'osoby_zorganizowana_wycieczka'
cursor.execute("DROP TABLE IF EXISTS osoby_zorganizowana_wycieczka;")
cursor.execute("""
CREATE TABLE osoby_zorganizowana_wycieczka (
    id_wycieczki INT UNSIGNED NOT NULL,
    id_osoby INT UNSIGNED NOT NULL,
    data_platnosci DATE NULL,
    ocena TINYINT NULL COMMENT 'Każdy klient może ocenić wycieczkę',
    PRIMARY KEY (id_wycieczki, id_osoby)
);
""")

# Usuwanie i utworzenie tabeli 'pracownicy'
cursor.execute("DROP TABLE IF EXISTS pracownicy;")
cursor.execute("""
CREATE TABLE pracownicy (
    id_pracownika SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    imie VARCHAR(30) NULL,
    nazwisko VARCHAR(50) NULL,
    id_stanowiska SMALLINT UNSIGNED NOT NULL,
    data_zatrudnienia DATE NOT NULL,
    czy_zatrudniony BOOL NOT NULL,
    numer_telefonu VARCHAR(18) NOT NULL,
    email VARCHAR(150) NOT NULL,
    PRIMARY KEY (id_pracownika)
);
""")

cursor.execute("ALTER TABLE pracownicy ADD CONSTRAINT UQ_id_pracownika UNIQUE (id_pracownika);")
cursor.execute("ALTER TABLE pracownicy ADD CONSTRAINT UQ_numer_telefonu UNIQUE (numer_telefonu);")
cursor.execute("ALTER TABLE pracownicy ADD CONSTRAINT UQ_email UNIQUE (email);")

# Usuwanie i utworzenie tabeli 'pracownicy_wycieczka'
cursor.execute("DROP TABLE IF EXISTS pracownicy_wycieczka;")
cursor.execute("""
CREATE TABLE pracownicy_wycieczka (
    id_wycieczki INT UNSIGNED NOT NULL,
    id_pracownika SMALLINT UNSIGNED NOT NULL,
    PRIMARY KEY (id_pracownika, id_wycieczki)
) COMMENT 'Jest w EKNF';
""")

# Usuwanie i utworzenie tabeli 'rodzaje'
cursor.execute("DROP TABLE IF EXISTS rodzaje;")
cursor.execute("""
CREATE TABLE rodzaje (
    id_rodzaju SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    nazwa VARCHAR(50) NOT NULL,
    PRIMARY KEY (id_rodzaju)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE rodzaje ADD CONSTRAINT UQ_id_rodzaju UNIQUE (id_rodzaju);")

# Usuwanie i utworzenie tabeli 'stanowiska'
cursor.execute("DROP TABLE IF EXISTS stanowiska;")
cursor.execute("""
CREATE TABLE stanowiska (
    id_stanowiska SMALLINT UNSIGNED NOT NULL AUTO_INCREMENT,
    stanowisko VARCHAR(40) NOT NULL,
    wynagrodzenie DECIMAL(10, 2) NOT NULL DEFAULT 4666,
    PRIMARY KEY (id_stanowiska)
) COMMENT 'Jest w EKNF';
""")

cursor.execute("ALTER TABLE stanowiska ADD CONSTRAINT UQ_id_stanowiska UNIQUE (id_stanowiska);")

# Usuwanie i utworzenie tabeli 'urlopy'
cursor.execute("DROP TABLE IF EXISTS urlopy;")
cursor.execute("""
CREATE TABLE urlopy (
    id_pracownika SMALLINT UNSIGNED NOT NULL,
    poczatek_urlopu DATE NOT NULL,
    koniec_urlopu DATE NOT NULL,
    PRIMARY KEY (id_pracownika, poczatek_urlopu)
) COMMENT 'Tabela zawierająca informacje o urlopie dla każdego pracownika w danym roku';
""")

# Usuwanie i utworzenie tabeli 'zorganizowana_wycieczka'
cursor.execute("DROP TABLE IF EXISTS zorganizowana_wycieczka;")
cursor.execute("""
CREATE TABLE zorganizowana_wycieczka (
    id_wycieczki INT UNSIGNED NOT NULL,
    id_oferty SMALLINT UNSIGNED NOT NULL,
    data_wycieczki DATE NULL COMMENT 'rozpoczęcie wycieczki',
    PRIMARY KEY (id_wycieczki)
);
""")

cursor.execute("ALTER TABLE zorganizowana_wycieczka ADD CONSTRAINT UQ_id_wycieczki UNIQUE (id_wycieczki);")

# Dodanie kluczy obcych 
cursor.execute("""
ALTER TABLE pracownicy ADD CONSTRAINT FK_stanowiska_TO_pracownicy
FOREIGN KEY (id_stanowiska)
REFERENCES stanowiska (id_stanowiska);
""")

cursor.execute("""
ALTER TABLE pracownicy_wycieczka ADD CONSTRAINT FK_pracownicy_TO_pracownicy_wycieczka
FOREIGN KEY (id_pracownika)
REFERENCES pracownicy (id_pracownika);
""")

cursor.execute("""
ALTER TABLE oferty ADD CONSTRAINT FK_rodzaje_TO_oferty
FOREIGN KEY (id_rodzaju)
REFERENCES rodzaje (id_rodzaju);
""")

cursor.execute("""
ALTER TABLE oferty ADD CONSTRAINT FK_kierunki_TO_oferty
FOREIGN KEY (id_kierunku)
REFERENCES kierunki (id_kierunku);
""")

cursor.execute("""
ALTER TABLE zorganizowana_wycieczka ADD CONSTRAINT FK_oferty_TO_zorganizowana_wycieczka
FOREIGN KEY (id_oferty)
REFERENCES oferty (id_oferty);
""")

cursor.execute("""
ALTER TABLE osoby ADD CONSTRAINT FK_dane_alarmowe_TO_osoby
FOREIGN KEY (id_alarm)
REFERENCES dane_alarmowe (id_alarm);
""")

cursor.execute("""
ALTER TABLE oferty ADD CONSTRAINT FK_cennik_TO_oferty
FOREIGN KEY (id_cennika)
REFERENCES cennik (id_cennika);
""")

cursor.execute("""
ALTER TABLE osoby_zorganizowana_wycieczka ADD CONSTRAINT FK_osoby_TO_osoby_zorganizowana_wycieczka
FOREIGN KEY (id_osoby)
REFERENCES osoby (id_osoby);
""")

cursor.execute("""
ALTER TABLE osoby_zorganizowana_wycieczka ADD CONSTRAINT FK_zorganizowana_wycieczka_TO_osoby_zorganizowana_wycieczka
FOREIGN KEY (id_wycieczki)
REFERENCES zorganizowana_wycieczka (id_wycieczki);
""")

cursor.execute("""
ALTER TABLE pracownicy_wycieczka ADD CONSTRAINT FK_zorganizowana_wycieczka_TO_pracownicy_wycieczka
FOREIGN KEY (id_wycieczki)
REFERENCES zorganizowana_wycieczka (id_wycieczki);
""")

cursor.execute("""
ALTER TABLE oferty ADD CONSTRAINT FK_miasta_TO_oferty
FOREIGN KEY (id_miasta)
REFERENCES miasta (id_miasta);
""")

cursor.execute("""
ALTER TABLE urlopy ADD CONSTRAINT FK_pracownicy_TO_urlopy
FOREIGN KEY (id_pracownika)
REFERENCES pracownicy (id_pracownika);
""")

# Zatwierdzenie zmian
conn.commit()

In [4]:
# dane_alarmowe
with open('../Dane do tabel/dane_alarmowe.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO dane_alarmowe (id_alarm,imie,nazwisko,numer_telefonu)
VALUES (%s, %s, %s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

#osoby
with open('../Dane do tabel/osoby.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO osoby (id_osoby,imie,nazwisko,numer_telefonu,email,id_alarm)
VALUES (%s, %s, %s, %s, %s, %s)
"""

cursor.executemany(sql, dane)
conn.commit()

# cennik
with open('../Dane do tabel/cennik.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO cennik (id_cennika,koszt_na_osobe,cena_biletu,koszt_podstawowy)
VALUES (%s, %s, %s, %s)
"""

cursor.executemany(sql, dane)
conn.commit()

# kierunki
with open('../Dane do tabel/kierunki.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO kierunki (id_kierunku,miejsce)
VALUES (%s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# miasta
with open('../Dane do tabel/miasta.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO miasta (id_miasta,nazwa)
VALUES (%s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# rodzaje
with open('../Dane do tabel/rodzaje.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO rodzaje (id_rodzaju,nazwa)
VALUES (%s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# oferty
with open('../Dane do tabel/oferty.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO oferty (id_oferty,nazwa,id_rodzaju,id_kierunku,id_miasta,opis,id_cennika,czas_trwania,czy_bezpieczna)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# zorganizowana_wycieczka
with open('../Dane do tabel/zorganizowana_wycieczka.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO zorganizowana_wycieczka (id_wycieczki,id_oferty,data_wycieczki)
VALUES (%s, %s, %s)
"""

cursor.executemany(sql, dane)
conn.commit()

# osoby_zorganizowana_wycieczka
with open('../Dane do tabel/osoby_zorganizowana_wycieczka.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        temp = list(row)
        while True:
            if temp[-1] == '':
                temp = temp[:-1]
            else:
                break
            
        dane.append(tuple(temp))

curr_len = 0

sql = """
INSERT INTO osoby_zorganizowana_wycieczka (id_wycieczki,id_osoby,data_platnosci,ocena) 
VALUES 
"""
j = 0
for dana in dane:
    if j != 0:
        sql += f", {dana}"[:-1]
    else:
        sql += f" {dana}"[:-1]
    for i in range(4 - len(dana)):
        sql += ", NULL"
    sql += ")"
    j += 1
    if j == 1000:
        j = 0
        cursor.execute(sql)
        sql = """
        INSERT INTO osoby_zorganizowana_wycieczka (id_wycieczki,id_osoby,data_platnosci,ocena) 
        VALUES 
        """
        
if j != 0:
    cursor.execute(sql)
        
conn.commit()

# stanowiska
with open('../Dane do tabel/stanowiska.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO stanowiska (id_stanowiska,stanowisko,wynagrodzenie)
VALUES (%s, %s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# pracownicy
with open('../Dane do tabel/pracownicy.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO pracownicy (id_pracownika,imie,nazwisko,id_stanowiska,data_zatrudnienia,czy_zatrudniony,numer_telefonu,email)
VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# urlopy
with open('../Dane do tabel/pracownik_urlop.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO urlopy (id_pracownika,poczatek_urlopu,koniec_urlopu)
VALUES (%s, %s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()

# pracownicy_wycieczka
with open('../Dane do tabel/pracownicy_wycieczka.csv', newline='', encoding='utf-8') as csvfile:
    reader = csv.reader(csvfile)
    dane = []
    
    # Pomiń nagłówek, jeśli jest
    next(reader)  # jeśli plik CSV ma nagłówek, usuń ten krok, jeśli go nie ma
    
    # Zbiera dane do listy jako krotki
    for row in reader:
        dane.append(tuple(row))


sql = """
INSERT INTO pracownicy_wycieczka (id_wycieczki, id_pracownika)
VALUES (%s, %s)
"""
cursor.executemany(sql, dane)
conn.commit()


# Zamykam połączenie
cursor.close()
conn.close()

print("Zrobione")

Zrobione
